In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

In [2]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200731


import datetime 
base = datetime.date(2020,2,1)
for i in range(0,10): 
    d=base + datetime.timedelta(i)
    d=d.strftime("%Y%m%d")
    print(f"N029kakutokuruiseki{d}.csv")

from datetime import datetime, timedelta

def date_range(start_date: datetime, end_date: datetime):
    diff = (end_date - start_date).days + 1
    return (start_date + timedelta(i) for i in range(diff))

In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [142]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202007_取引価額")

#input
new_cust = "NEW_CUST_取引価額.csv"
uri_tot = "SHUKEI_TOT.csv"

#out_put
result = "年間利用金額帯別新規数"

print(new_cust)
print(uri_tot)
print(result)

NEW_CUST_取引価額.csv
SHUKEI_TOT.csv
年間利用金額帯別新規数


column_names = ['実績月','入会申込番号','直近３ケ月買物利用金額','直近３ケ月全体利用金額','年令','性別',
                '都道府県名','顧客状況コ－ド','マイペ設定','DM希望店','カード券種名','カ－ド加入日','初回利用日',
                'あとからリボ申込有無コード','Weｂ明細有無コード','解約','入会チャネル']

新規入会客の集計

In [93]:
df = pd.read_csv(data_dir /r"input"/f"{new_cust}",encoding='shift-jis')
df.head()

,cp,no,kanyu_yymmdd,card,channel,kzk_f
0,20180331,4980006342739,20180301,ゴールド,店頭タブレット,1
1,20180331,4980006342748,20180301,ゴールド,店頭タブレット,0
2,20180331,4980006342757,20180301,ゴールド,店頭タブレット,0
3,20180331,4980006342766,20180301,ゴールド,郵送,1
4,20180331,4980006342775,20180301,ゴールド,郵送,1


In [94]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(141347, 6)
RangeIndex(start=0, stop=141347, step=1)
Index(['cp', 'no', 'kanyu_yymmdd', 'card', 'channel', 'kzk_f'], dtype='object')
cp               int64
no               int64
kanyu_yymmdd     int64
card            object
channel         object
kzk_f            int64
dtype: object


In [95]:
df['kanyu_yymmdd'] = pd.to_datetime(df['kanyu_yymmdd'], format = '%Y%m%d')
df['kanyu_year'] = df['kanyu_yymmdd'].dt.year
df['kanyu_month'] = df['kanyu_yymmdd'].dt.month
df['kanyu_yymm'] = df[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df.head()

,cp,no,kanyu_yymmdd,card,channel,kzk_f,kanyu_year,kanyu_month,kanyu_yymm
0,20180331,4980006342739,2018-03-01,ゴールド,店頭タブレット,1,2018,3,2018年3月
1,20180331,4980006342748,2018-03-01,ゴールド,店頭タブレット,0,2018,3,2018年3月
2,20180331,4980006342757,2018-03-01,ゴールド,店頭タブレット,0,2018,3,2018年3月
3,20180331,4980006342766,2018-03-01,ゴールド,郵送,1,2018,3,2018年3月
4,20180331,4980006342775,2018-03-01,ゴールド,郵送,1,2018,3,2018年3月


In [96]:
df['card'].unique()

array(['ゴールド', '一般', '外商'], dtype=object)

In [97]:
df['kanyu_yymm'].unique()

array(['2018年3月', '2018年4月', '2018年5月', '2018年6月', '2018年7月', '2018年8月',
       '2018年9月', '2018年10月', '2018年11月', '2018年12月', '2019年1月',
       '2019年2月', '2019年3月'], dtype=object)

In [98]:
#noの重複がないかチェック
print(len(df) == len(df['no'].unique()))

True


In [99]:
#重複削除
df_dup = df.drop_duplicates(subset=['no'])
df_dup.shape[0]

141347

In [100]:
#欠損データがないかチェック
df_dup.isnull().sum()

cp              0
no              0
kanyu_yymmdd    0
card            0
channel         0
kzk_f           0
kanyu_year      0
kanyu_month     0
kanyu_yymm      0
dtype: int64

集計

In [101]:
#客数
df_new= df_dup.groupby(['kanyu_yymm','card'])['no'].count()
df_new.head()

kanyu_yymm  card
2018年10月    ゴールド      462
            一般       9712
            外商       1214
2018年11月    ゴールド      485
            一般      11210
Name: no, dtype: int64

売上の集計

In [105]:
df = pd.read_csv(data_dir /r"input"/f"{uri_tot}",encoding='shift-jis')
df.head()

,kanyu_yymm,card,tot_amt_class,no_of_new_account
0,201803,ゴールド,21_100万円以上,36
1,201803,ゴールド,12_10万円以上,74
2,201803,ゴールド,3_1万円以上,28
3,201803,ゴールド,22_200万円以上,7
4,201803,ゴールド,13_20万円以上,63


In [106]:
#2019年3月までに限定する
df =df[(df["kanyu_yymm"] <=201903)]

In [107]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(977, 4)
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            967, 968, 969, 970, 971, 972, 973, 974, 975, 976],
           dtype='int64', length=977)
Index(['kanyu_yymm', 'card', 'tot_amt_class', 'no_of_new_account'], dtype='object')
kanyu_yymm            int64
card                 object
tot_amt_class        object
no_of_new_account     int64
dtype: object


In [108]:
df['kanyu_yymmdd']=df['kanyu_yymm']*100+1
df['kanyu_yymmdd'] = pd.to_datetime(df['kanyu_yymmdd'], format = '%Y%m%d')
df['kanyu_year'] = df['kanyu_yymmdd'].dt.year
df['kanyu_month'] = df['kanyu_yymmdd'].dt.month
df['kanyu_yymm'] = df[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df.head()

,kanyu_yymm,card,tot_amt_class,no_of_new_account,kanyu_yymmdd,kanyu_year,kanyu_month
0,2018年3月,ゴールド,21_100万円以上,36,2018-03-01,2018,3
1,2018年3月,ゴールド,12_10万円以上,74,2018-03-01,2018,3
2,2018年3月,ゴールド,3_1万円以上,28,2018-03-01,2018,3
3,2018年3月,ゴールド,22_200万円以上,7,2018-03-01,2018,3
4,2018年3月,ゴールド,13_20万円以上,63,2018-03-01,2018,3


In [109]:
df['card'].unique()

array(['ゴールド', '一般', '外商'], dtype=object)

In [110]:
df['kanyu_yymm'].unique()

array(['2018年3月', '2018年4月', '2018年5月', '2018年6月', '2018年7月', '2018年8月',
       '2018年9月', '2018年10月', '2018年11月', '2018年12月', '2019年1月',
       '2019年2月', '2019年3月'], dtype=object)

集計

In [113]:
#客数
df_uri = df.groupby(['kanyu_yymm','card','tot_amt_class'])['no_of_new_account'].sum()
df_uri.head()

kanyu_yymm  card  tot_amt_class
2018年10月    ゴールド  10_8万円以上          9
                  11_9万円以上          9
                  12_10万円以上        63
                  13_20万円以上        38
                  14_30万円以上        36
Name: no_of_new_account, dtype: int64

In [114]:
df_uri.head()

kanyu_yymm  card  tot_amt_class
2018年10月    ゴールド  10_8万円以上          9
                  11_9万円以上          9
                  12_10万円以上        63
                  13_20万円以上        38
                  14_30万円以上        36
Name: no_of_new_account, dtype: int64

Merge

In [126]:
df_uri.head()

kanyu_yymm  card  tot_amt_class
2018年10月    ゴールド  10_8万円以上          9
                  11_9万円以上          9
                  12_10万円以上        63
                  13_20万円以上        38
                  14_30万円以上        36
Name: no_of_new_account, dtype: int64

In [121]:
df_new.head()

kanyu_yymm  card
2018年10月    ゴールド      462
            一般       9712
            外商       1214
2018年11月    ゴールド      485
            一般      11210
Name: no, dtype: int64

In [144]:
df_merge = pd.merge(df_uri, df_new, left_on=['kanyu_yymm', 'card'], right_index=True)
df_merge.head()

no_of_new_account   no
kanyu_yymm card tot_amt_class                        
2018年10月   ゴールド 10_8万円以上                       9  462
                11_9万円以上                       9  462
                12_10万円以上                     63  462
                13_20万円以上                     38  462
                14_30万円以上                     36  462

In [141]:
df_merge = df_merge.rename(columns={'no_of_new_account':'no_of_account','no':'new_account'})
df_merge.head()

account  new_account
kanyu_yymm card tot_amt_class                      
2018年10月   ゴールド 10_8万円以上             9          462
                11_9万円以上             9          462
                12_10万円以上           63          462
                13_20万円以上           38          462
                14_30万円以上           36          462

In [143]:
df_merge.to_csv(data_dir /r"output"/f"{result}.csv", header=True,encoding='utf-8-sig')